# **Lab 1: Matrix Factorization**
**Lovisa Strange**

# **Abstract**

In this lab report, three algorithms related to matrix vector multiplication and matrix factorization are presented.

#**About the code**

A short statement on who is the author of the file, and if the code is distributed under a certain license.

In [ ]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2024 Lovisa Strange (lstrange@kth.se)

# This file is part of the course DD2365 Advanced Computation in Fluid Mechanics
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them.

In [1]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np

#try:
#    from dolfin import *; from mshr import *
#except ImportError as e:
#    !apt-get install -y -qq software-properties-common
#    !add-apt-repository -y ppa:fenics-packages/fenics
#    !apt-get update -qq
#    !apt install -y --no-install-recommends fenics
#    from dolfin import *; from mshr import *

#import dolfin.common.plotting as fenicsplot

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

# **Introduction**

When doing computations with large matrices, the time complexity of the computations are important. One area in which this can be considered is when working with sparse matrices, ie matrices that largely consist of zeros. Often, computations with such matrices can become faster if we consider the large amount of 0:s in them.

Another way to compute things more efficiently is by factorizing maricies into a form that is easier to invert. When solving a matrix equation $$Ax = b$$, it is expensive to directly invert the matrix A. However, if we can write it as $$A = BC,$$ where $B$ and $C$ is easier to invert, we can solve the equation more efficiently.

In this report, an algorithm for multiplicating a sparse matrix and a vector will be presented. Also, a way to factorize matrices will be presented, as well as using the factorization to solve a matrix equation.

# **Method**

##Sparse matrix-vector product
For matrices that are sparse it is often not efficient to use regular matrix-vector multiplication. Instead, we can use another algoritm that uses a specific data structure to make the computation more efficient.

This data structure is called *compressed row structure* (lecture X, slide Y) or CRS. A matrix is represented as 3 arrays. The first one conatins all non-zero elements of the matrix: The seccond one (of the same length) has the column index for each of the non-zero elements. Lastly, the third array contains the index that points to the position in the first array where each new row starts.

We look at an example using the matrix
$$A= \begin{bmatrix}3&0&0\\0&2&0 \end{bmatrix}.$$
This matrix would be reprecented with the values $$
val=[3,2],$$
the column indicies $$
col\_idx = [0,1]
$$
and the row pointers $$
row\_ptr=[0,1,2].
$$

We can find the CRS representation of a given matrix by looping over its rows and saving the non-zero elements and their columns, as well as keeping track of when a new row is started. This is done in the code below.

In [2]:
# CRS representation of A

class CRS:
  def __init__(self,A):
    val = []
    col_idx = []
    row_ptr = []

    for row in A:
      colCounter = 0
      row_ptr.append(len(val))

      for elem in row:
        if elem != 0:
          val.append(elem)
          col_idx.append(colCounter)
        colCounter +=1
    row_ptr.append(len(val))


    self.val = val
    self.col_idx = col_idx
    self.row_ptr = row_ptr

Then, we can use this data structure to compute the product between a sparse matrix and a vector more efficiently. (Lecture X, slide Y) To understand how this works, we will lok at an example.

Using the regular matrix-vector product, we would do the following computations
$$c =A\cdot b = \begin{bmatrix}3&0&0\\0&2&0 \end{bmatrix} \begin{bmatrix}1\\2\\3 \end{bmatrix}= \begin{bmatrix}3⋅1+0\cdot2+0⋅3\\0⋅1+2⋅2+0⋅3 \end{bmatrix} = \begin{bmatrix}3\\4 \end{bmatrix}$$

If we instead store the matrix using CRS, we loop over each row, and then look at the non-zero elements of that row and multiply them with the corresponding element in the vector. We get for the same A as above that for the first row we compute
$$
c[0] = val[0]⋅b[0]= 3⋅1 =3
$$

and for the seccond row we get

$$
c[1] = val[1]⋅b[1] =2⋅2=4.
$$

This means that we don't have to compute the multiplication with the zeroes, which is useful for sparse matrices.

The algorithm for this is presented below.

In [3]:
# Sparse matrix vector multiplucation

## Input: vector x with length n, CRS representation of mxn matrix A
## Output: b = Ax

def sparse_matrix_vector_product(A,x):

  b= np.empty(len(x)) # result
  for i in range(len(x)):
    b[i] = 0
    for j in range(A.row_ptr[i],A.row_ptr[i+1]):
      b[i] += A.val[j]*x[A.col_idx[j]]

  return b

##QR factorization

One way in which we can factorize matrices is by QR-factorization, $$
A = QR,
$$
where Q is an orthogonal matrix and R is an upper triangular matrix. This can be done by so called Gram-Schmidt QR factorization. (Reference! kap 5.3)

We want to construct an orthogonal basis for the matrix A using Gram Schmits method, which is done by going through all the vectors $a_{:j}$ that span $A,$ and suptracting the projection of $a_{:j}$ on all vectors alredy in the orthogonal base from $a_{:j}.$ This is done by computing $$
v_j = a_j - ∑_{i=1}^{j-1}(a_{:j}q_i)q_i
$$
and normalising the result. Writing this as a linear system with a matric R having elements $r_{ij} = (a_{:j}q_i)$ and $r_{ii}=||v_j||, $ we get an upper triangular matrix R, and together with the (orthogonal) columns of Q, this is a QR-factorization of A.

However, when doing this nummerically, instead want to use an equivalent form of Gram-Schmidts method, where we construct a projector $P_j$ as $$
P_J = P^{\perp q_{:j-1}}...P^{\perp q_{:1}}
$$ where $$
P^{\perp q_{:i}} = I - q_{:i}q_{:i}^T.
$$
This is done in the algorithm below

In [4]:
# QR - factrorization (based on algorithm 5.3 from the course book, p.89)

## Input: Matrix A
## Output: Q orthogonal, R upper triangular so that A = QR

def QR_factorization(A):
  n = A.shape[0]
  R= np.zeros([n,n])
  Q= np.zeros([n,n])
  v = np.zeros(n)

  for j in range(n):
    v[:] = A[:, j]
    for i in range(j):
      R[i,j] = np.dot(Q[:,i],v[:])
      v[:] = v[:] - R[i,j]*Q[:,i]
    R[j,j] = np.sqrt(sum([elem**2 for elem in v]))
    Q[:,j] = v[:]/R[j,j]

  return Q,R

##Solving Ax=b

Now, we want to use QR-factorisation to solve the matrix equation $$Ax = b \implies x = A^{-1}b.$$ By factorizing the matrix A, we get A = QR, and the equation can be written as $$x = (QR)^{-1}b = Q^{-1}R^{-1}b.$$ Since both Q and R are of a specific form, we can find the inverse to them more easily than for the matrix A, which we don't know the structure of.

We know that Q is a normal matrix, and so its inverse is the same as its transpose, $$Q^T = Q^{-1}.$$ We also know that R is upper triangular, which we can use to solve the equation $$Rx=b$$ one row at a time. This is done by doing so called backwards substitution, where we start from the last row of the matrix and solve for $x_{n-1},$ then use that value to solve for $x_{n-2}$ using the next to last row and so on, until all $x_j$:s have been solved for. The code for the backwards substitution is shown below.

In [13]:
# Backward substitution

## Input: Upper triangular matrix U, vector b
## Output: solution to Ux = b

def backward_substitution(U,b):
  n = len(b)
  x = np.zeros(n)
  x[n-1] = b[n-1]/U[n-1,n-1]
  for i in reversed(range(n-1)):
    sum = 0
    for j in range(i+1, n):
      sum += U[i,j]*x[j]
    x[i] = (b[i]-sum)/U[i,i]

  return x

Then, we simply use the QR-factorization from the previous section, and solve the system given by $$Rx = b$$ using backwards substitution. Finally, we find the inverse of $Q,$ and multiply these together to get the final result.  

In [14]:
# Solving Ax = b

## Input: matrix A, vector b
## Output: Solution b

def solver(A,b):
  Q,R = QR_factorization(A) # x = (QR)^(-1)b = Q^(-1)R^(-1)b

  partialSol = backward_substitution(R,b)
  QInverse = np.transpose(Q)

  x = np.matmul(QInverse,partialSol)

  return x



# **Results**
In this section, the results produced by the algorithms in the previous section are presented

##Sparse matrix-vector product
Here, we want to test the computations made by the sparse matrix-vector product with that of regular matrix-vector multiplication. If these are the same, the computations are correct.

We look at a vector $x$ and a matrix $A$, and compute the CRS representation of the matrix, and then use the algorithm **sparse_matrix_vector_product** from above to compute the resulting vector.

Then, we use the **matmul-**function from the numpy package to compute the regular matrix vector product.

In [15]:
x = np.array([1,1,1,1,1,1])
A = np.array([[3,2,0,2,0,0],[0,2,1,0,0,0],[0,0,1,0,0,0],[0,0,3,2,0,0],[0,0,0,0,1,0],[0,0,0,0,2,3]])
A_sparse = CRS(A)

b_sparse = sparse_matrix_vector_product(A_sparse,x)
print("sparse = ",np.array(b_sparse))

b_dense = np.matmul(A,x)
print("regular = ",b_dense)
print("sparse - regular = ", b_dense-b_sparse)

sparse =  [7. 3. 1. 5. 1. 5.]
regular =  [7 3 1 5 1 5]
sparse - regular =  [0. 0. 0. 0. 0. 0.]


We can see that the two methods give the same result. We can also verify this by computing the difference between the two vectors, and we see that it is equal to the zero vector. This result also holds for different input vectors and matrices.

##QR factorization

We now want to verify our result from the QR-factorization. We want to see if R is upper triangular, which is easy to verify by looking at it.

We can also check that multiplying Q and R gives us the matrix A back. Another way of checking this is by computing the matrix norm of  $$
QR-A.
$$
If this is 0, the matrices are the same.

Another check can be done by looking at $$
Q^TQ,
$$
which should be equal to the identity matrix $I,$ as $Q^T$ is the inverse of $Q.$ Therefor, it follows that the matrix norm if $$
Q^TQ - I
$$
should be 0.

In [16]:
A = np.array([[2, -1],[-1,2]])
Q, R = QR_factorization(A)

print("Q = ", Q)
print("R = ", R)
print("A = QR = ", np.matmul(Q,R))
print()
QTQ =  np.matmul(np.transpose(Q),Q)
I = np.identity(A.shape[0])
print("||Q^TQ - I||_F = ", np.linalg.norm(QTQ - I,'fro'))

QR = np.matmul(Q,R)
print("||QR - A||_F = ", np.linalg.norm(QR - A,'fro'))

Q =  [[ 0.89442719  0.4472136 ]
 [-0.4472136   0.89442719]]
R =  [[ 2.23606798 -1.78885438]
 [ 0.          1.34164079]]
A = QR =  [[ 2. -1.]
 [-1.  2.]]

||Q^TQ - I||_F =  2.6901577681355055e-16
||QR - A||_F =  0.0


Looking at the results, we can verify that R is upper triangular, since it has 0 below the diagonal. Also, we can see that $$||QR-A||$$ is equal to 0, so the factorisation gives us the original matrix back. Finally, computing
$$
||Q^TQ - I||
$$
gives a answer very close to 0. Considering the presicion with which these computations are done, this confirms that the matrix $Q^T$ is the inverse of $Q,$ and so $Q$ is orthogonal. These results holds for other matrices A as well.


##Solving Ax = b


Nowm we want to verify that the solver algorithm gives a correct answer. We can check that the norm of $$
Ax-b
$$
is equal to 0, since if the solution is correct, it should hold that $$
Ax = b
$$
We can also construct a solution by solving the system by hand, and comparing that solution, $y,$ to the one found by the algorithm, $x$. We can compute the norm of $$x-y$$ to se if these are equal to each other.    

In [18]:
A = np.array([[1, 3],[0,1]])
b = np.array([1,1])

x = solver(A,b)
y = np.array([-2,1]) # Computed by hand

print("|Ax-b| = ",np.linalg.norm(np.matmul(A,x)-b))
print("|x-y| = ",np.linalg.norm(x-y))

|Ax-b| =  0.0
|x-y| =  0.0


We can see that $Ax-b$ has length zero, so the found solution satisfies the equation. We can also see that it is equal to the solution computed by hand. This holds for other vectors $b$ and matrices $A$ as well.

# **Discussion**

The methods presented in this report had the expected outcomes. For the sparse matrix-vector product, the difference in preformance is not really noticible for the size of matrix used. If a much larger matrix would have been used, a more noticable difference would have beem seen. Also, the built in numpy matrix multiplication already has some optimisations buit in, so a more comparable regular matrix product would have been to implement the matrix multiplication from scratch as well.

For the QR-factorization and solving of the $Ax=b$ equation also had the expected outcome. For one of the computations that should have equaled zero, a number the order of magnitude $10^{-16}$ was computed. However, this is to be expected, since computations are not done with infinite precision, and round off errors can result in differences between the expected and actual result.